In [70]:
from gensim import models
import numpy as np
from numpy.core.fromnumeric import shape
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,Conv1D,MaxPooling1D
from keras.layers import LSTM,Dropout
from keras.layers import TimeDistributed ,Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
import regex as re
import pandas as pd
from nltk.tokenize import RegexpTokenizer
import nltk
from gensim.models import Word2Vec
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import make_blobs
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from keras import backend as K
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [71]:
data =  pd.read_csv('labeled_data.csv')
print(data.size,type(data))
labels = data.iloc[::,1]
data = data.iloc[::,0]
print(data)
print(data.size)
print(labels.shape[0])
noMentionData = []

file = open("stopwordslist.txt","r")
stopwords_list = []
for lines in file:
    lines = lines.rstrip(lines[-1])
    stopwords_list.append(lines)


#Pre-requisites for every model
voc_size = 10000
# truncate and pad input sequences
# max_length = 300
# x_train = sequence.pad_sequences(x_train, maxlen=max_length)
# x_test = sequence.pad_sequences(x_test, maxlen=max_length)


49564 <class 'pandas.core.frame.DataFrame'>
0        !!!!! RT @mleew17: boy dats cold...tyga dwn ba...
1        !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
2        !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
3        !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
4        !!!!!!!!!!!!!!!!!!"@T_Madison_x: The shit just...
                               ...                        
24777    you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24778    you've gone and broke the wrong heart baby, an...
24779    young buck wanna eat!!.. dat nigguh like I ain...
24780                youu got wild bitches tellin you lies
24781    ~~Ruffled | Ntac Eileen Dahlia - Beautiful col...
Name: !!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out..., Length: 24782, dtype: object
24782
24782


In [72]:
#Preprocessing
def preprocess(tweet):  
    # removal of @name[mention]
    tweet_name = tweet.str.replace('@[\w\-]+', '')

    # removal of links[https://abc.com]
    tweets = tweet_name.str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]'
            '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '')
    
    # removal of punctuations and numbers
    punc_remove = tweets.str.replace(r'[^\w\s]',"")
    # remove whitespace with a single space
    newtweet = punc_remove
    # remove leading and trailing whitespace
    newtweet=newtweet.str.replace(r'^\s+|\s+?$','')
    # replace normal numbers with numbr
    newtweet=newtweet.str.replace(r'\d+(\.\d+)?','numbr')
    return newtweet

In [73]:
def getFinalList(newtweet):
    final_list=[]
    # filter(None, stopwords_list)
    for text in newtweet:
        words = [word for word in str(text).split() if word.lower() not in stopwords_list]
        text = " ".join(words)
        final_list.append(text)
    all_words = [nltk.word_tokenize(sent) for sent in final_list]
    max_length = 0
    for sentence in all_words:
        if len(sentence) > max_length:
            max_length = len(sentence)
    print(max_length)
    return all_words


In [74]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [88]:
def simple_lstm(embedding_layer,voc_size, max_length, x_train, y_train, x_test, y_test):
    
    #Word vector matrix
    embedding_vector_features=32
    #Defining a sequential model
    model=Sequential()
    
    
    #Defining input and Embedding layer
    # model.add(Embedding(input_dim=x_train.syn0.shape[0], output_dim=model.syn0.shape[1], weights=[model.syn0]))
    model.add(embedding_layer)
   
    #1st LSTM hidden layer with 128 neurons and ReLu activation function
    model.add(LSTM(64,activation='sigmoid',return_sequences=True))
    #Dropout layer with 20% dropout frequency
    model.add(Dropout(0.5))
    #2nd LSTM hidden layer with 32 neurons and ReLu activation function
    # model.add(LSTM(32,activation='relu'))
    #Dropout layer with 20% dropout frequency
    # model.add(Dropout(0.2))
    #3rd Dense hidden layer with 4 neurons and ReLu activation function
    # model.add(Dense(4,activation='relu'))
    #Dropout layer with 20% dropout frequency
    # model.add(Dropout(0.2))
    #Output Layer with 1 neuron for 2 different categories and sigmoid activation function
    # model.add(Dense(1,activation='sigmoid'))
    model.add(TimeDistributed(Dense(1, activation='sigmoid')))
    #Compiling the model
    
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc'])
    #model.build()
 
    
    #Will print the summary of the model
    
    
 
    print(model.summary())
    print("x_train: ",x_train.shape)
    print("y_train: ",y_train.shape)
    #Training the model
    model.fit(x_train,y_train,epochs=1,batch_size=256)
    #Evaluating the model
    accuracy= model.evaluate(x_test, y_test, verbose=0)
    # accuracy = model.evaluate(x_test, y_test)
    #Printing the accuracy
    # y_pred = model.predict(x_test).round()
    y_pred = model.predict_on_batch(x_test).round().argmax(1)
    print(y_pred.shape,y_test.shape)
    print('Accuracy: %.2f' % (accuracy[1]*100))
    print('F1 score: %.2f' % f1_score(y_test, y_pred, average='micro'))
    print('Precision: %.2f' % average_precision_score(y_test, y_pred, average="micro"))
    return model
    

In [76]:
#CNN LSTM
def cnn_lstm_model(embedding_layer,x_train, y_train, x_test, y_test):
    # define the sequential model
    model = Sequential()
    
    model.add(embedding_layer)
    model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(LSTM(350))
    model.add(Dense(1, activation='sigmoid'))


    # compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    model.build()
    #Will print the summary of the model
    print(model.summary())
    
    # fit the model on the dataset
    model.fit(x_train, y_train, epochs=10, batch_size=128)
    # evaluate the model
    accuracy = model.evaluate(x_test, y_test, verbose=0)
    # accuracy = model.evaluate(x_test, y_test)
    # Printing the accuracy
    # print('Loss: %.2f' % (loss))

    y_pred = model.predict(x_test).round()
    print('Accuracy: %.2f' % (accuracy[1]*100))
    print('F1 score: %.2f' % f1_score(y_test, y_pred, average='micro'))
    print('Precision: %.2f' % average_precision_score(y_test, y_pred, average="micro"))
    # print('Recall: %.2f' % (recall))
   # Confusion Matrix
    # print(y_pred, y_pred.round())
    # cfm = confusion_matrix(y_test, y_pred.round())
    # print(cfm)
    # y_pred_bool = np.argmax(y_pred.round(), axis=1)
    # print(classification_report(y_test, y_pred_bool))
    
    return model



In [86]:
#Simple Neural Network
def cnn_model(embedding_layer,x_train, y_train, x_test, y_test):
    # define the sequential model
    model = Sequential()
    
    model.add(embedding_layer)
    model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))


    # compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    model.build()
    #Will print the summary of the model
    print(model.summary())
    
    # fit the model on the dataset
    model.fit(x_train, y_train, epochs=20, batch_size=128)
    # evaluate the model
    # evaluate the model
    accuracy= model.evaluate(x_test, y_test, verbose=0)
    # accuracy = model.evaluate(x_test, y_test)
    #Printing the accuracy
    y_pred = model.predict_on_batch(x_test).round().argmax(1)
    ny =[]
    for x in y_pred:
      ny.append([x[0]])
    print(ny)
    print("================\n\n",y_pred,"=================\n\n",y_pred.shape,y_test.shape,y_test)
    print('Accuracy: %.2f' % (accuracy[1]*100))
    print('F1 score: %.2f' % f1_score(y_test, y_pred, average='micro'))
    print('Precision: %.2f' % average_precision_score(y_test, y_pred, average="micro"))
    return model

In [78]:
#Simple Neural Network
def simple_Neural_network(x_train, y_train, max_length):
    # define the sequential model
    model = Sequential()
    
    #Adding input layer and first hidden layer with 128 neurons and ReLu activation function
    model.add(Dense(128, input_dim=max_length, activation='relu'))
    model.add(Dropout(0.2))
    #2nd hidden layer with 32 neurons and ReLu activation function
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    #3rd hidden layer with 4 neurons and ReLu activation function
    model.add(Dense(32, activation='relu'))
    #Output layer with 1 neuron and sigmoid activation function
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    model.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics=['acc'])
    model.build()
    #Will print the summary of the model
    print(model.summary())
    
    # fit the model on the dataset
    model.fit(x_train, y_train, epochs=80, batch_size=32)
    # evaluate the model
    # evaluate the model
    accuracy = model.evaluate(x_test, y_test, verbose=0)
    # accuracy = model.evaluate(x_test, y_test)
    #Printing the accuracy
    y_pred = model.predict(x_test).round()
    print('Accuracy: %.2f' % (accuracy[1]*100))
    print('F1 score: %.2f' % f1_score(y_test, y_pred, average='micro'))
    print('Precision: %.2f' % average_precision_score(y_test, y_pred, average="micro"))
    return model

In [79]:
#  newtweet = preprocess(data)
# all_words = getFinalList(newtweet)
# # print(all_words)
# w2v_model = Word2Vec(size=32, window=4, min_count=2, workers=8)
# w2v_model.build_vocab(all_words)
# words = w2v_model.wv.vocab.keys()
# vocab_size = len(words)
# # print("Vocab size", vocab_size)

# w2v_model.train(all_words, total_examples=len(all_words), epochs=7)
# print(w2v_model.wv.most_similar("love"))


# tokenizer = Tokenizer()
# # print(newtweet.iloc[0])
# # print(newtweet[0])
# newtweet=newtweet.astype(str)
# tokenizer.fit_on_texts(newtweet)
# vocab_size = len(tokenizer.word_index) + 1
# print("Total words", vocab_size)

# x_train, x_test, y_train, y_test = train_test_split(newtweet, labels, test_size=0.33, random_state=42)
# x_train = sequence.pad_sequences(tokenizer.texts_to_sequences(x_train), maxlen=300)
# x_test = sequence.pad_sequences(tokenizer.texts_to_sequences(x_test), maxlen=300)


# encoder = LabelEncoder()
# encoder.fit(labels.tolist())

# y_train = encoder.transform(y_train.tolist())
# y_test = encoder.transform(y_test.tolist())

# y_train = y_train.reshape(-1,1)
# y_test = y_test.reshape(-1,1)
# print("x_train", x_train.shape)
# print("x_test", x_test.shape)
# print("y_train",y_train.shape)
# print("y_test",y_test.shape)

# embedding_matrix = np.zeros((vocab_size, 32))
# for word, i in tokenizer.word_index.items():
#   if word in w2v_model.wv:
#     embedding_matrix[i] = w2v_model.wv[word]
# print(embedding_matrix.shape)

# embedding_layer = Embedding(vocab_size, 32, weights=[embedding_matrix], input_length=300, trainable=False)
# max_length=300
# #Simple LSTM Model

# # model=simple_lstm(embedding_layer,voc_size, max_length, x_train, y_train, x_test, y_test)
# # model=simple_Neural_network(x_train, y_train, max_length)
# model = cnn_lstm_model(embedding_layer,x_train, y_train, x_test, y_test)
# # model=cnn_model(embedding_layer,x_train, y_train, x_test, y_test)



In [89]:
 newtweet = preprocess(data)
all_words = getFinalList(newtweet)
# print(all_words)
w2v_model = Word2Vec(size=32, window=4, min_count=2, workers=8)
w2v_model.build_vocab(all_words)
words = w2v_model.wv.vocab.keys()
vocab_size = len(words)
# print("Vocab size", vocab_size)

w2v_model.train(all_words, total_examples=len(all_words), epochs=7)
print(w2v_model.wv.most_similar("love"))


tokenizer = Tokenizer()
# print(newtweet.iloc[0])
# print(newtweet[0])
newtweet=newtweet.astype(str)
tokenizer.fit_on_texts(newtweet)
vocab_size = len(tokenizer.word_index) + 1
print("Total words", vocab_size)

x_train, x_test, y_train, y_test = train_test_split(newtweet, labels, test_size=0.33, random_state=42)
x_train = sequence.pad_sequences(tokenizer.texts_to_sequences(x_train), maxlen=300)
x_test = sequence.pad_sequences(tokenizer.texts_to_sequences(x_test), maxlen=300)


encoder = LabelEncoder()
encoder.fit(labels.tolist())

y_train = encoder.transform(y_train.tolist())
y_test = encoder.transform(y_test.tolist())

y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)
print("x_train", x_train.shape)
print("x_test", x_test.shape)
print("y_train",y_train.shape)
print("y_test",y_test.shape)

embedding_matrix = np.zeros((vocab_size, 32))
for word, i in tokenizer.word_index.items():
  if word in w2v_model.wv:
    embedding_matrix[i] = w2v_model.wv[word]
print(embedding_matrix.shape)

embedding_layer = Embedding(vocab_size, 32, weights=[embedding_matrix], input_length=300, trainable=False)
max_length=300
#Simple LSTM Model

model=simple_lstm(embedding_layer,voc_size, max_length, x_train, y_train, x_test, y_test)
# model=simple_Neural_network(x_train, y_train, max_length)
# model = cnn_lstm_model(embedding_layer,x_train, y_train, x_test, y_test)
# model=cnn_model(embedding_layer,x_train, y_train, x_test, y_test)



26
[('loyal', 0.9950183033943176), ('numbr', 0.9941142201423645), ('Yall', 0.9938414096832275), ('trust', 0.9898370504379272), ('Side', 0.9898181557655334), ('em', 0.9891805052757263), ('man', 0.9890669584274292), ('dat', 0.9887652397155762), ('dick', 0.9881954193115234), ('ah', 0.9876891374588013)]
Total words 26031
x_train (16603, 300)
x_test (8179, 300)
y_train (16603, 1)
y_test (8179, 1)
(26031, 32)
Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_20 (Embedding)    (None, 300, 32)           832992    
                                                                 
 lstm_6 (LSTM)               (None, 300, 64)           24832     
                                                                 
 dropout_17 (Dropout)        (None, 300, 64)           0         
                                                                 
 time_distributed_3 (TimeDis  (None, 300, 

In [ ]:
# data =  pd.DataFrame(["happy","bitch","best","lol","Preeti is good","This is not offensive","no offence","black bitch","anay"])
# data =  pd.read_csv('pred.csv')
# rawdata = list(data.iloc[::,0])
# data = data.iloc[::,0]
# noMentionData = []
# file = open("stopwordslist.txt","r")
# stopwords_list = []
# for lines in file:
#     lines = lines.rstrip(lines[-1])
#     stopwords_list.append(lines)
# newtweet = preprocess(data)
# all_words = getFinalList(newtweet)

# tokenizer = Tokenizer()

# newtweet=newtweet.astype(str)
# tokenizer.fit_on_texts(newtweet)

# data = sequence.pad_sequences(tokenizer.texts_to_sequences(data), maxlen=300)


# yans = (model.predict(data).round())
# print(rawdata)
# for x in range(yans.shape[0]):
#   print(rawdata[x],yans[x])
  # print()


# New Section

# New Section